In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
from nltk.stem import SnowballStemmer
stemmer_eng = SnowballStemmer('english')
stemmer_rus = SnowballStemmer('russian')

In [4]:
df = pd.read_csv("data/train.csv")

In [5]:
df.head(5)

,id,url,title,target
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False


## Посчитаем сколько всего в сумме слов в названиях

In [6]:
df['title'].map(lambda x: len(x.split(' '))).sum()

1423026

## Подгружаем стоп-слова, которые могут создавать шум

In [7]:
file = open("stopwords/eng_rus.txt", "r")
stopwords_eng_rus = [line[:-1] for line in file]

In [8]:
stopwords_eng_rus[:20]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 'her',
 'hers']

## Почистим названия: оставим только буквы (все слова в нижнем регистре, стоп-слова удалены)

## Плюс стеммер

In [9]:
%%time
def Cleaner(string):
    string = string.lower()
    string = re.sub(r"\s+" ," ", re.sub(r"[^a-zа-яё]", " ", string))

    string = string.split(' ')
    tokens = []
    for word in string:
        if (word in stopwords_eng_rus) or word=='':
            continue
        else: 
            tokens.append(word)
            
    for i in range (len(tokens)):
        tokens[i] = stemmer_eng.stem(tokens[i])
        tokens[i] = stemmer_rus.stem(tokens[i])
        
    return tokens

df['title_tokens'] = df['title'].map(lambda x: Cleaner(x))

CPU times: user 2min 16s, sys: 338 ms, total: 2min 16s
Wall time: 2min 16s


In [10]:
df.head(5)

,id,url,title,target,title_tokens
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False,"[экс, министр, экономик, молдов, глав, мидэ, ц..."
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False,"[песн, стал, известн, мног, телезрител, благод..."
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False,"[банш, сезон, сер, брем, красот, смотрет, онлайн]"
3,3,colorbox.spb.ru,Не Беси Меня Картинки,False,"[бес, картинк]"
4,4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,False,"[новомосковск, сыгра, следж, хоккеист, алексин..."


## Cнова посчитаем сколько всего в сумме слов в "новых" названиях

In [11]:
df['title_tokens'].map(lambda x: len(x)).sum()

1068786

## Выберем все адреса сайтов, с которых есть 3 и более запросов и все эти запросы порнография

In [13]:
black_list = []
for sub_df in df.groupby('url'):
    
    if ( sub_df[1].target.count() > 2 ):
        if ( all(sub_df[1].target) ):
            
            black_list.append(sub_df[0])

In [14]:
len(black_list)

951

In [15]:
df['in_black_list'] = df.url.map(lambda x: 1 if x in black_list else 0)

In [16]:
df.in_black_list.value_counts()

0    123867
1     11442
Name: in_black_list, dtype: int64

# Признак - слово "порно" в названии

In [17]:
tmp = []
for title in df['title'].values:
    if (' порно ' in title.lower()) or ('porn' in title.lower()):
        tmp.append(1)
    else:
        tmp.append(0)

In [18]:
df['bad_title'] = tmp

In [19]:
df.bad_title.value_counts()

0    127577
1      7732
Name: bad_title, dtype: int64

In [20]:
df.head(3)

,id,url,title,target,title_tokens,in_black_list,bad_title
0,0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",False,"[экс, министр, экономик, молдов, глав, мидэ, ц...",0,0
1,1,www.kp.by,Эта песня стала известна многим телезрителям б...,False,"[песн, стал, известн, мног, телезрител, благод...",0,0
2,2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,False,"[банш, сезон, сер, брем, красот, смотрет, онлайн]",0,0


In [21]:
df.to_csv("data/clean_test.csv", index=0)